### Imports

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import joblib
import random

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

sns.set_style("whitegrid")

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.6.0


### Directories

In [2]:
cwd = os.getcwd()
print("Current working directory:", cwd)

Current working directory: /workspaces/ML_Project_Image_Recognition/Jupyter_Notebooks


In [3]:
os.chdir('/workspaces/ML_Project_Image_Recognition')
print("New working directory set.")

New working directory set.


In [4]:
work_dir = os.getcwd()
work_dir

'/workspaces/ML_Project_Image_Recognition'